In [25]:
import pandas as pd
import os
import graphein
import pickle
path = '../datasets/ALPHAFOLD PDBs/'
import numpy as np
from graphein.protein.visualisation import plotly_protein_structure_graph
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.graphs import construct_graph
from Bio.PDB import PDBParser   
from utils import *

## create a df with mutation name and path to che .pdb file

In [26]:

pdb_files = []
for folder in os.listdir(path):
    # if the folder is a directory
    if os.path.isdir(path + folder):
        # for every file in the folder
        for file in os.listdir(path + folder):
            if 'rank_001' in file and file.endswith('.pdb'):
                pdb_files.append(path + folder + '/' + file)



In [27]:
# for each pdb file in pdb_files, get name of the mutation which is the name of the folder before '_'
mutations = []
for pdb in pdb_files:
    mutations.append(pdb.split('/')[-2].split('_')[0])

pdb_files.append('../datasets/ALPHAFOLD PDBs/H371fs_9b3dd/H371fs_9b3dd_unrelaxed_rank_001_alphafold2_ptm_model_3_seed_000.pdb')
mutations.append('His371Profs')

pdb_files.append('../datasets/ALPHAFOLD PDBs/K353Q_ed31e/K353Q_ed31e_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb')
mutations.append('L353Q')

# create a dataframe with the mutation and the pdb file

df = pd.DataFrame({'mutation': mutations, 'pdb_file': pdb_files})

out_path = "../datasets/pdb_files.csv"



In [28]:
# search the index of Ala218 in the df

df['mutation'] = df['mutation'].replace('Ala218', '(p.(Ala218_Asn219insLysIle))')
df['mutation'] = df['mutation'].replace('E168', 'E168*')
df['mutation'] = df['mutation'].replace('G115Mfs', 'G115Mfs*')
df['mutation'] = df['mutation'].replace('G372', 'G372_P373delinsA')
df['mutation'] = df['mutation'].replace('R321', 'R321*')
df['mutation'] = df['mutation'].replace('W60', 'W60*')
# add a row with the mutation and the pdb file



In [29]:
df.to_csv(out_path, index=False)

## Parse the .pdb files

In [ ]:
structures = {}
parser = PDBParser()
for i, row in df.iterrows():
    structures[row['mutation']] = parser.get_structure(row['mutation'], row['pdb_file'])
 

## Create graphs from the .pdb files

In [ ]:


config = ProteinGraphConfig()
(config.dict())

In [ ]:
graphs = {}
for i, row in df.iterrows():
    print(row['mutation'])
    graphs[row['mutation'] ] = construct_graph(path = row['pdb_file'], config= config, )

In [ ]:
graph_original = graphein.protein.graphs.construct_graph(path ='../datasets/ALPHAFOLD PDBs/HGD_normal.pdb', config= config )
structure_original = parser.get_structure('HGD_normal', '../datasets/ALPHAFOLD PDBs/HGD_normal.pdb')

In [ ]:
from graphein.protein.visualisation import plotly_protein_structure_graph

p = plotly_protein_structure_graph(
    graphs['G161R'],
    colour_edges_by="kind",
    colour_nodes_by="degree",
    label_node_ids=False,
    plot_title="Peptide backbone graph. Nodes coloured by degree.",
    node_size_multiplier=1
    )
p.show()

In [ ]:
import networkx as nx
# check if graphs are isomorphic
G1 = graphs['G161R']
G2 = graphs['G170A']

nx.is_isomorphic(G1, G2)

In [ ]:
# plot structures['G161R'] 

import nglview as nv
view = nv.show_biopython(structures['G161R'])
view

In [ ]:
# save the graphs
os.makedirs('../datasets/graphs/', exist_ok=True)
for i, graph in enumerate(graphs):
    mut = df.iloc[i]['mutation']
    with open(f'../datasets/graphs/graph_{mut}.gpickle', 'wb') as f:
        pickle.dump(graph, f)

## Associate to each patient its mutations

In [ ]:
import pandas as pd
df_patients =pd.read_excel('../datasets/aku_prin_v2.0.xlsx')

In [ ]:
df_patients = df_patients[['Protein change allele 1 ', 'Protein change allele 2']]

In [ ]:
df_patients['graph_allele1'] = [graphs[mut] if mut in graphs else None for mut in df_patients['Protein change allele 1 '] ]
df_patients['graph_allele2'] = [graphs[mut] if mut in graphs else None for mut in df_patients['Protein change allele 2'] ]
df_patients['structure_allele1'] = [structures[mut] if mut in structures else None for mut in df_patients['Protein change allele 1 '] ] 
df_patients['structure_allele2'] = [structures[mut] if mut in structures else None for mut in df_patients['Protein change allele 2'] ]

In [ ]:
#show all rows
pd.set_option('display.max_columns', 500)
df_patients

In [ ]:
df_patients.to_csv('../datasets/aku_prin_v2.0_with_graphs.csv', index=False)



In [ ]:
#read the csv file
df_patients_2 = pd.read_csv('../datasets/aku_prin_v2.0_with_graphs.csv')


In [ ]:
graph_test = df_patients_2['graph_allele1'][0]
graph_test_og = df_patients['graph_allele1'][0]

In [ ]:
type(graph_test_og)

In [ ]:
import networkx as nx

def string_to_graph(input_string):
    # Initialize a NetworkX graph
    G = nx.Graph()

    # Split the input string into lines
    lines = input_string.strip().split('\n')

    # Iterate through each line
    for line in lines:
        # Split the line into nodes and edges
        nodes = line.strip().split()
        # Assuming the first element is a node and the rest are its neighbors
        node = nodes[0]
        neighbors = nodes[1:]

        # Add the node to the graph
        G.add_node(node)

        # Add edges between the node and its neighbors
        for neighbor in neighbors:
            G.add_edge(node, neighbor)

    return G

In [ ]:
type(graph_test)
# convert to a scipy sparse matrix
graph = string_to_graph(graph_test)
type(graph)

In [ ]:
from torch_geometric.data import Data


In [ ]:
from utils import create_graph_df
df_p = create_graph_df()

## Convert to pytorch geometric

In [ ]:
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.edges.distance import *
from graphein.protein.graphs import construct_graph

edge_fns = [
    add_aromatic_interactions,
    add_hydrophobic_interactions,
    add_aromatic_sulphur_interactions,
    add_cation_pi_interactions,
    add_disulfide_interactions,
    add_hydrogen_bond_interactions,
    add_ionic_interactions,
    add_peptide_bonds
    ]
config = ProteinGraphConfig(edge_construction_functions=edge_fns)

g = construct_graph(config=config, path= df['pdb_file'][0])
g2 = construct_graph(config=config, path='../datasets/ALPHAFOLD PDBs/HGD_normal.pdb')

In [56]:
p = plotly_protein_structure_graph(
    g.subgraph([1,2,3,4]),
    colour_edges_by="kind",
    colour_nodes_by="degree",
    label_node_ids=False,
    plot_title="Peptide backbone graph. Nodes coloured by degree.",
    node_size_multiplier=1
    )
p.show()

ValueError: max() arg is an empty sequence

In [16]:
# check if g and g2 are isomorphic
import networkx as nx

nx.is_isomorphic(g, g2)

False

In [17]:
contact_map = get_contact_map(structures['G161R'])

NameError: name 'get_contact_map' is not defined

In [ ]:
contact_map_original = get_contact_map(structure_original)

In [ ]:
for i in range(len(contact_map)):
    for j in range(len(contact_map)):
        if contact_map[i][j] != contact_map_original[i][j]:
            print(i,j, contact_map[i][j], contact_map_original[i][j])

In [ ]:
import matplotlib.pyplot as plt
# 0 -> white, 1 -> black
# invert white and black
#contact_map = 1 - contact_map
#contact_map_original = 1 - contact_map_original
fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(contact_map)
ax[1].imshow(contact_map_original)

In [52]:
g.edges(data=True)

OutEdgeDataView([('A:MET:1', 'A:LEU:164', {'kind': {'hydrophobic'}, 'distance': 10.518538586704903}), ('A:MET:1', 'A:PHE:193', {'kind': {'hydrophobic'}, 'distance': 12.366443708681976}), ('A:MET:1', 'A:ALA:2', {'kind': {'peptide_bond'}, 'distance': 3.880878379954722}), ('A:ALA:2', 'A:MET:1', {'kind': {'peptide_bond'}, 'distance': 3.880878379954722}), ('A:ALA:2', 'A:GLU:3', {'kind': {'peptide_bond'}, 'distance': 3.8727523804137016}), ('A:GLU:3', 'A:ALA:2', {'kind': {'peptide_bond'}, 'distance': 3.8727523804137016}), ('A:GLU:3', 'A:LEU:4', {'kind': {'peptide_bond'}, 'distance': 3.8747165057588413}), ('A:LEU:4', 'A:TYR:6', {'kind': {'hydrophobic'}, 'distance': 6.6816707491465035}), ('A:LEU:4', 'A:LEU:173', {'kind': {'hydrophobic'}, 'distance': 7.551629029553822}), ('A:LEU:4', 'A:LEU:164', {'kind': {'hydrophobic'}, 'distance': 10.371771497675795}), ('A:LEU:4', 'A:GLU:3', {'kind': {'peptide_bond'}, 'distance': 3.8747165057588413}), ('A:LEU:4', 'A:LYS:5', {'kind': {'peptide_bond'}, 'distance

In [1]:
g = g.to_directed()

from dgl import from_networkx
g_dgl=from_networkx(g, node_attrs=['residue_number', 'coords'], edge_attrs=['distance'])
g_dgl.edata['distance'][1]

NameError: name 'g' is not defined